In [66]:
import pandas as pd

from surprise import Dataset, Reader
from surprise import KNNBasic, KNNWithMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error, f1_score

In [2]:
path = "../data/AllSites_SiteOccurrences_AllGenera_26.1.24.csv"

df = pd.read_csv(path)
df.head()

,SITE_NAME,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,...,Total_Gen_Count,Large_GenCount,Small_GenCount,smallperlarge,smallprop,Herb_GenCount,Nonherb_GenCount,DietRatio,HerbProp,mid_age
0,Aba Zawei,1,1,1,1,0,0,0,0,0,...,4,4,0,0.0,0.000000,4,0,NaN,1.000000,0.0265
1,Abric Romani,1,0,1,0,1,1,1,1,1,...,12,12,0,0.0,0.000000,6,5,1.2,0.500000,0.0550
2,Acheng_Jiaojie,0,0,0,0,0,0,1,0,0,...,7,5,2,0.4,0.285714,5,2,2.5,0.714286,0.2100
3,Adler cave,1,0,0,0,0,1,0,1,0,...,10,5,5,1.0,0.500000,6,4,1.5,0.600000,0.0275
4,Adyrgan,1,0,0,1,0,0,0,0,0,...,11,5,6,1.2,0.545455,11,0,NaN,1.000000,2.2000


## Building content-based filtering

In [3]:
def build_site_genus_matrix(df):
    cols_redundant = ['LAT',
    'LONG',
    'ALTITUDE',
    'MAX_AGE',
    'BFA_MAX',
    'BFA_MAX_ABS',
    'MIN_AGE',
    'BFA_MIN',
    'BFA_MIN_ABS',
    'COUNTRY',
    'age_range',
    'Total_Gen_Count',
    'Large_GenCount',
    'Small_GenCount',
    'smallperlarge',
    'smallprop',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'DietRatio',
    'HerbProp',
    'mid_age'
    ]

    df_site_genus = df.drop(columns=cols_redundant).set_index('SITE_NAME')

    return df_site_genus

In [4]:
def build_site_info(df):
    site_info_cols = [
    'SITE_NAME',
    'LAT',
    'LONG',
    'MAX_AGE',
    'MIN_AGE',
    'age_range',
    'Large_GenCount',
    'Small_GenCount',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'mid_age'
    ]

    df_site_info = df[site_info_cols].set_index('SITE_NAME')

    return df_site_info

In [5]:
def build_genus_info(df):
    site_genus = build_site_genus_matrix(df)
    site_genus = site_genus.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})
    site_info = build_site_info(df)

    genus_info = site_genus.merge(site_info, on="SITE_NAME", how="left")
    genus_info = genus_info[genus_info["presence"] == 1]

    genus_info = genus_info.drop(["SITE_NAME", "presence"], axis=1)
    genus_info = genus_info.groupby('genus').mean().reset_index().set_index("genus")

    return genus_info

In [6]:
def get_recommendations_for_site(genus_info, site_name, site_indices, genus_site_similarity_matrix, num_recommend = 10):
    idx = site_indices[site_name]

    # Sorted similarity scores
    sim_scores = sorted(list(enumerate(genus_site_similarity_matrix[:,idx])), key=lambda x: x[1], reverse=True)

    # Get the scores of the num_recommend most similar sites
    similar_genus_for_site = sim_scores[:num_recommend]

    # Get the genus indices
    genus_indices = [i[0] for i in similar_genus_for_site]
    genus_site_similarities = [i[1] for i in similar_genus_for_site]

    recommended_genus = genus_info.iloc[genus_indices].index.to_frame(index=False).assign(similarity=genus_site_similarities)
    recommended_genus.insert(0, "SITE_NAME", site_name)

    return recommended_genus

In [14]:
def normalize_columns_min_max(df):
    return (df - df.min()) / (df.max() - df.min())

In [15]:
def normalize_columns_mean(df):
    return (df - df.mean()) / df.std()

In [20]:
def find_recommendations_for_all_sites(df, n_species_to_recommend, normalization: None):
    genus_info = build_genus_info(df)
    site_info = build_site_info(df)
    
    if normalization != None:
        genus_info = normalization(genus_info)
        site_info = normalization(site_info)

    site_indices = pd.Series(df.index, index=df["SITE_NAME"]).drop_duplicates()
    sim = cosine_similarity(genus_info, site_info)

    recommendations = []
    for site, idx in site_indices.items():
        site_recommendations = get_recommendations_for_site(
            genus_info=genus_info,
            site_name=site,
            site_indices=site_indices,
            genus_site_similarity_matrix=sim,
            num_recommend=n_species_to_recommend                        
        )

        recommendations.append(site_recommendations)
    
    return pd.concat(recommendations).reset_index(drop=True)

In [71]:
all_site_recommendations = find_recommendations_for_all_sites(df, 10, normalization=normalize_columns_min_max)
all_site_recommendations_matrix = pd.pivot(all_site_recommendations, index="SITE_NAME", columns="genus", values="similarity").fillna(0)
all_site_recommendations_matrix.head()

genus,Acinonyx,Aepyosciurus,Aeretes,Ailuropoda,Ailurus,Alactagulus,Alcelaphus,Alces,Algarolutra,Alilepus,...,Villanyia,Viverra,Viverravus,Viverricula,Vormela,Vulpes,Wushanomys,Xenocyon,Yangia,Zygolophodon
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Abric Romani,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.926382,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Acheng_Jiaojie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Adler cave,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.954767,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Adyrgan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.985751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.970305,0.0


### Performance

In [72]:
# Find missing columns from recommendations

df_site_genus = build_site_genus_matrix(df)
missing_columns = df_site_genus.columns.difference(all_site_recommendations_matrix.columns)

for col in missing_columns:
    all_site_recommendations_matrix[col] = 0

In [73]:
column_order = df_site_genus.columns
all_site_recommendations_matrix = all_site_recommendations_matrix[column_order]
all_site_recommendations_matrix

genus,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,Homo,...,Euarctos,Paracervulus,Eostyloceros,Cervocerus,Antispiroides,Sinoryx,Prospalax,Pliopetaurista,Predicrostonyx,Boocercus
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,0.0,0.944944,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abric Romani,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acheng_Jiaojie,0.0,0.975505,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adler cave,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adyrgan,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuurland (-42 to -46 m),0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zverinogolovskoe,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kudaro 3 cave l.3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
all_site_recommendations_matrix[all_site_recommendations_matrix > 0.5] = 1


In [81]:
rms = root_mean_squared_error(df_site_genus, all_site_recommendations_matrix)
mae = mean_absolute_error(df_site_genus, all_site_recommendations_matrix)
f1 = f1_score(df_site_genus, all_site_recommendations_matrix, zero_division=1.0, average='macro')
print(rms, mae, f1)

0.1864828321767479 0.0479083984519437 0.1386704617611954


## Building collaborative filtering

In [26]:
def fit_knn_collaborative_filtering(df, site_based=True, similarity="cosine", k=5, min_k=1):
    site_genus_matrix = build_site_genus_matrix(df)
    df_site_genus = site_genus_matrix.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})

    reader = Reader(rating_scale=(0, 1))
    data = Dataset.load_from_df(df_site_genus, reader) # Column order must be user, item, rating

    sim_options = {
    'name': similarity,
    'user_based': site_based  # True for user-user, False for item-item
    }

    trainset = data.build_full_trainset()

    knn = KNNBasic(k=k, min_k=min_k, sim_options=sim_options)
    knn.fit(trainset)

    return knn, trainset

In [27]:
knn, trainset = fit_knn_collaborative_filtering(
    df, 
    site_based=True, 
    similarity="cosine",
    k=5,
    min_k=1
)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [32]:
def predict_collaborative_filtering(algotihm, trainset):
    # Testset equal to trainset
    testset = trainset.build_testset()

    # Get predictions for all user-item pairs
    predictions = algotihm.test(testset)

    # Get item scores from the predictions
    item_scores = [(prediction.uid, prediction.iid, prediction.est) for prediction in predictions]
    return pd.DataFrame(item_scores, columns =['SITE_NAME', 'PREDICTED_GENUS', 'SCORE'])

In [33]:
predictions = predict_collaborative_filtering(knn, trainset)

In [34]:
display(predictions.head(10))

,SITE_NAME,PREDICTED_GENUS,SCORE
0,Aba Zawei,Equus,1.000000
1,Aba Zawei,Coelodonta,0.891040
2,Aba Zawei,Bos,0.596936
3,Aba Zawei,Gazella,0.585506
4,Aba Zawei,Ursus,0.041709
5,Aba Zawei,Vulpes,0.042974
6,Aba Zawei,Cervus,0.458731
7,Aba Zawei,Canis,0.267233
8,Aba Zawei,Sus,0.073484
9,Aba Zawei,Homo,0.044935


In [82]:
items_scores_matrix_knn = pd.pivot(predictions, index="SITE_NAME", columns="PREDICTED_GENUS", values="SCORE")
items_scores_matrix_knn.head()

PREDICTED_GENUS,Acinonyx,Aepyosciurus,Aeretes,Ailuropoda,Ailurus,Alactagulus,Alcelaphus,Alces,Algarolutra,Alilepus,...,Villanyia,Viverra,Viverravus,Viverricula,Vormela,Vulpes,Wushanomys,Xenocyon,Yangia,Zygolophodon
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,0.020855,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.042974,0.0,0.000000,0.0,0.027979
Abric Romani,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.199330,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.665286,0.0,0.022387,0.0,0.000000
Acheng_Jiaojie,0.066017,0.0,0.0,0.0,0.0,0.000000,0.0,0.295206,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.043909,0.433696,0.0,0.000000,0.0,0.000000
Adler cave,0.023809,0.0,0.0,0.0,0.0,0.023528,0.0,0.090624,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.780527,0.0,0.027585,0.0,0.000000
Adyrgan,0.040802,0.0,0.0,0.0,0.0,0.000000,0.0,0.046691,0.0,0.0,...,0.185094,0.0,0.0,0.0,0.000000,0.118985,0.0,0.000000,0.0,0.023630


### Performance

In [83]:
column_order = df_site_genus.columns
items_scores_matrix_knn = items_scores_matrix_knn[column_order]
items_scores_matrix_knn

PREDICTED_GENUS,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,Homo,...,Euarctos,Paracervulus,Eostyloceros,Cervocerus,Antispiroides,Sinoryx,Prospalax,Pliopetaurista,Predicrostonyx,Boocercus
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,1.000000,0.891040,0.596936,0.585506,0.041709,0.042974,0.458731,0.267233,0.073484,0.044935,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Abric Romani,0.902037,0.142833,0.585015,0.023502,0.856823,0.665286,0.977613,0.838192,0.709099,0.622239,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Acheng_Jiaojie,0.494266,0.214607,0.259920,0.090081,0.609590,0.433696,1.000000,0.477943,0.655624,0.231977,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022109
Adler cave,0.906084,0.358209,0.119465,0.044288,0.550833,0.780527,0.497553,0.798726,0.286545,0.170797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Adyrgan,0.904407,0.350904,0.183218,0.811326,0.087110,0.118985,0.313011,0.273502,0.083317,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuurland (-42 to -46 m),0.329670,0.000000,0.016297,0.017742,0.273457,0.244811,0.239478,0.263016,0.134879,0.016049,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Zverinogolovskoe,0.213616,0.084544,0.042292,0.024892,0.044659,0.040662,0.061420,0.042480,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
kudaro 3 cave l.3,0.688617,0.220385,0.239831,0.020745,0.932404,0.868828,0.954196,0.826506,0.378347,0.295807,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [87]:
items_scores_matrix_knn[items_scores_matrix_knn > 0.5] = 1
items_scores_matrix_knn[items_scores_matrix_knn <= 0.5] = 0

In [88]:
items_scores_matrix_knn

PREDICTED_GENUS,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,Homo,...,Euarctos,Paracervulus,Eostyloceros,Cervocerus,Antispiroides,Sinoryx,Prospalax,Pliopetaurista,Predicrostonyx,Boocercus
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abric Romani,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acheng_Jiaojie,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adler cave,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adyrgan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuurland (-42 to -46 m),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zverinogolovskoe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kudaro 3 cave l.3,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
rms = root_mean_squared_error(df_site_genus, items_scores_matrix_knn)
mae = mean_absolute_error(df_site_genus, items_scores_matrix_knn)
f1 = f1_score(df_site_genus, items_scores_matrix_knn, zero_division=1.0, average='macro')
print(rms, mae, f1)

0.12665790485301573 0.02972859713560283 0.09974169664645385


## Hybrid algorithm content-knn